In [1]:
from pydantic import BaseModel, Field
from typing import List
from langchain_groq import ChatGroq

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model_name_openai = "gpt-4o-2024-08-06" # Garante 100% do output em JSON

llm_openai = ChatOpenAI(
    model=model_name_openai, # 100% json output
    temperature=0,
)

system_prompt = """
Você é um assistente de IA muito prestativo que vai auxiliar um cozinheiro a classificar textos que falam especificamente de receitas de BOLOS. Você precisa classificar o texto como YES ou NO, dependendo se o texto fala de um bolo ou não.
"""

prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt), 
            ("human", "query do usuário: \n\n {query}")
        ]
)

class GetSchema(BaseModel):
    """Schema de bolo"""
    
    resultado: str = Field(description="YES caso o texto fale de BOLOS e NO caso contrário", examples=['YES', 
                                                                                                       'NO'])
    

llm_openai_with_tools_extraction = llm_openai.bind_tools([GetSchema]) #, strict=True)
chain_openai_structured_extraction = prompt | llm_openai_with_tools_extraction

In [2]:
query_1 = "Olá, eu gostaria de saber como fazer um bolo de cenoura com cobertura de chocolate. Pode me ajudar?"
query_2 = "Olá, eu queria saber qual foi o placar do jogo de ontem. Pode me informar?"
response_1 = chain_openai_structured_extraction.invoke({"query": query_1})
response_2 = chain_openai_structured_extraction.invoke({"query": query_2})

In [4]:
response_1.tool_calls

[{'name': 'GetSchema',
  'args': {'resultado': 'YES'},
  'id': 'call_rDlrL6zqJuMTJrYdg0HbcjWw',
  'type': 'tool_call'}]

In [5]:
response_2.tool_calls

[{'name': 'GetSchema',
  'args': {'resultado': 'NO'},
  'id': 'call_5xYU0mcvA52pxlGRwO3GWa4q',
  'type': 'tool_call'}]